In [2]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
from numba import njit

In [4]:
data = vbt.BinanceData.pull(
    symbols=['BTCUSDT'],
    start='2020-01-01 UTC',
    end='2021-01-01 UTC',
    timeframe="15m",
)

  0%|          | 0/1 [00:02<?, ?it/s, symbol=BTCUSDT]

4it [00:02,  1.58it/s, 2020-01-01 00:00 → 2020-02-11 16:00]

In [10]:
@vbt.parameterized(
    merge_func="concat"
)
def rsi_pipeline(data, upper_threshold=70, lower_threshold=30):
    rsi = vbt.RSI.run(close=data).rsi
    entries = rsi.vbt.crossed_below(lower_threshold)
    exits = rsi.vbt.crossed_above(upper_threshold)
    pf = vbt.Portfolio.from_signals(
        data,
        long_entries=entries,
        long_exits=exits,
    )

    if len(pf.trades) < 1:
        return vbt.NoResult

    return pf.sharpe_ratio

In [12]:
grid = rsi_pipeline(
    data.close,
    vbt.Param(range(60,90,1), name="upper"),
    vbt.Param(range(20,40,1), name="lower")
)

 18%|#8        | 108/600 [00:02<00:09, 53.62it/s, upper=65, lower=28]

In [13]:
grid.sort_values()

upper  lower  symbol 
60     20     BTCUSDT   -1.071379
       21     BTCUSDT   -1.052097
61     20     BTCUSDT   -0.948015
       21     BTCUSDT   -0.931126
62     20     BTCUSDT   -0.827772
                           ...   
89     37     BTCUSDT    2.286922
       34     BTCUSDT    2.297431
       35     BTCUSDT    2.299347
       32     BTCUSDT    2.335433
       33     BTCUSDT    2.335433
Name: sharpe_ratio, Length: 600, dtype: float64

In [14]:
data2 = vbt.BinanceData.pull(
    symbols=['BTCUSDT'],
    start='2021-01-01 UTC',
    end='2022-01-01 UTC',
    timeframe="15m",
)

  0%|          | 0/1 [00:02<?, ?it/s, symbol=BTCUSDT]

4it [00:02,  1.59it/s, 2021-01-01 00:00 → 2021-02-11 16:15]

In [16]:
rsi_pipeline(
    data2.close,
    89,
    33
)

symbol
BTCUSDT    0.869423
Name: sharpe_ratio, dtype: float64

In [19]:
vbt.Portfolio.from_holding(data.close).sharpe_ratio

symbol
BTCUSDT    2.200665
Name: sharpe_ratio, dtype: float64